### **Import Libraries**

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import seaborn as sns

### **Dataset Configuration**

In [ ]:
dataset_dir = 'batik_jogja'
batch_size = 32
img_size = (224, 224)
EPOCHS = 100

### **Load Dataset**

In [ ]:
print("Loading dataset...")

dataset = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    image_size=img_size,
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=123
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    image_size=img_size,
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=123
)

class_names = dataset.class_names
print("Kelas ditemukan:", class_names)

### **Build EfficientNetB0 Model**

In [ ]:
print("Building model...")

base_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights='imagenet',
    input_shape=img_size + (3,),
    pooling='avg'
)

base_model.trainable = False

inputs = tf.keras.Input(shape=img_size + (3,))
x = tf.keras.applications.efficientnet.preprocess_input(inputs)
x = base_model(x)
outputs = tf.keras.layers.Dense(len(class_names), activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

### **Callbacks Setup**

In [ ]:
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=10,
        min_lr=1e-7,
        verbose=1
    ),
    ModelCheckpoint(
        'best_batik_model.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

### **Start Model Training**

In [ ]:
print("Starting training...")

history = model.fit(
    dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    callbacks=callbacks
)

### **Save Final Model**

In [ ]:
model_path = 'batik_model_jogja_final.h5'
model.save(model_path)
print(f"✅ Model disimpan sebagai '{model_path}'")

### **Model Evaluation**

#### **1. Classification Report**

In [ ]:
print("\n=== CLASSIFICATION REPORT ===")
y_true = []
y_pred = []

for batch_images, batch_labels in val_dataset:
    preds = model.predict(batch_images)
    y_true.extend(batch_labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

print(classification_report(y_true, y_pred, target_names=class_names))

#### **2. Training History Visualization**

In [ ]:
print("\n=== TRAINING HISTORY ===")
plt.figure(figsize=(15, 5))

# Accuracy
plt.subplot(1, 3, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy', color='blue')
plt.plot(history.history['val_accuracy'], label='Val Accuracy', color='red')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Loss
plt.subplot(1, 3, 2)
plt.plot(history.history['loss'], label='Train Loss', color='blue')
plt.plot(history.history['val_loss'], label='Val Loss', color='red')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# Learning Rate
plt.subplot(1, 3, 3)
if 'lr' in history.history:
    plt.plot(history.history['lr'], label='Learning Rate', color='green')
    plt.title('Learning Rate')
    plt.xlabel('Epoch')
    plt.ylabel('Learning Rate')
    plt.legend()
    plt.grid(True)
else:
    plt.text(0.5, 0.5, 'Learning Rate\nNot Available',
             horizontalalignment='center', verticalalignment='center',
             transform=plt.gca().transAxes)
    plt.title('Learning Rate')

plt.tight_layout()
plt.show()

#### **3. Confusion Matrix**

In [ ]:
print("\n=== CONFUSION MATRIX ===")
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.tight_layout()
plt.show()

# Additional Metrics
print(f"\nTotal Samples: {len(y_true)}")
print(f"Correct Predictions: {np.sum(np.array(y_true) == np.array(y_pred))}")
print(f"Overall Accuracy: {np.sum(np.array(y_true) == np.array(y_pred)) / len(y_true) * 100:.2f}%")

### **Image Prediction Function**

In [ ]:
def predict_image(model, img_path, class_names):
    """
    Prediksi gambar batik baru.
    
    Args:
        model: Model terlatih
        img_path: Path ke gambar
        class_names: Daftar nama kelas
    """
    try:
        img = image.load_img(img_path, target_size=img_size)
        img_array = image.img_to_array(img)
        img_array_exp = tf.expand_dims(img_array, 0)
        img_array_preprocessed = tf.keras.applications.efficientnet.preprocess_input(img_array_exp)

        predictions = model.predict(img_array_preprocessed)
        predicted_class = np.argmax(predictions[0])
        confidence = predictions[0][predicted_class] * 100

        # Display image with prediction
        plt.figure(figsize=(8, 6))
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"Predicted: {class_names[predicted_class]} ({confidence:.2f}%)")
        plt.show()

        # Print prediction
        print("\n=== HASIL KLASIFIKASI ===")
        print(f"Motif Terdeteksi: {class_names[predicted_class]}")
        print(f"Confidence: {confidence:.2f}%")

        # Print all probabilities
        print("\nProbabilitas semua kelas:")
        for i, class_name in enumerate(class_names):
            prob = predictions[0][i] * 100
            print(f"{class_name}: {prob:.2f}%")

    except Exception as e:
        print(f"Error: {e}")
        print("Pastikan file gambar ada dan dapat dibaca.")

In [ ]:
img_path = 'batik_ceplok_0162.jpg'
if os.path.exists(img_path):
    predict_image(model, img_path, class_names)
else:
    print(f"File {img_path} tidak ditemukan. Silakan ganti dengan path yang benar.")